In [45]:
import pandas as pd
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
import re

CSV_FILE_NAME = "data/data_Song_Lyrics_Gr6_2021-01-31_18-33.csv"

import csv
import pandas as pd

CSV_FILE_NAME = "../data/data_Song_Lyrics_Gr6_2021-01-31_18-33.csv"
GROUP_5_FNAME = "../data/Group_5_sentiart results with raw data.xlsx"
GROUP_7_FNAME = "../data/group7data.xlsx"
DROP_PARTICIPANTS = [6, 17, 32, 39, 40, 42, 46]


def read_sosci(fname):
    """read the data from the CSV"""
    with open(fname, "r", encoding="utf16") as csv_file:
        reader = csv.reader(csv_file, delimiter=",")
        header = next(reader)
        rows = [r for r in reader]
        df = pd.DataFrame(rows[1:], columns=header)

    return df


def read_group_5():
    df = pd.read_excel(GROUP_5_FNAME,
                           engine='openpyxl',
)
    df = df.iloc[:38, ]  # select only participant data
    sr = df.filter(regex="SR0")
    sn = df.filter(regex="SN0")
    return df

def read_group_7():
    df = pd.read_excel(GROUP_7_FNAME,
                           engine='openpyxl',
)
    df = df.iloc[:38, ]  # select only participant data
    sr = df.filter(regex="SR0")
    sn = df.filter(regex="SN0")
    return df




def read_bfi_raw():
    """read raw bfi responses"""
    df = read_sosci(CSV_FILE_NAME)
    df = df.drop(DROP_PARTICIPANTS, axis=0)
    bfi_df = df.filter(regex="(BF02_)0?([1-9]|10)$").astype("int32")  # leave out mysterious 11th BFI question
    return bfi_df


def read_ocean():
    """read bfi and transform into OCEAN scores"""
    bfi_df = read_bfi_raw()
    ocean_pos = bfi_df.iloc[:, [9, 7, 5, 1, 8]]  # positively poled items, ordered by OCEAN

    ocean_neg = bfi_df.iloc[:, [4, 2, 0, 6, 3]]  # negatively poled items, ordered by OCEAN
    ocean_neg_repoled = (ocean_neg * -1) + 6  # invert negatively poled items for averaging

    ocean_pos.columns = ["Openness", "Conscientiousness", "Extraversion", "Agreeableness", "Neuroticism"]
    ocean_neg_repoled.columns = ["Openness", "Conscientiousness", "Extraversion", "Agreeableness", "Neuroticism"]
    ocean = (ocean_pos + ocean_neg_repoled) / 2  # average neg and pos poled items

    return ocean


def read_language():
    """read language (native english, native german, bilingual, other) and language level (B2 or lower, C1,
    C2) responses """
    df = read_sosci(CSV_FILE_NAME)
    df = df.drop(DROP_PARTICIPANTS, axis=0)
    lang_df = df.filter(regex="D01[2, 6]").astype("int32")
    lang_df.columns = ["language", "level"]
    lang_df = lang_df.replace(to_replace=-9, value=float("nan"))  # -9 indicates missing response

    return lang_df

   
df = read_sosci(CSV_FILE_NAME)
df = df.drop(DROP_PARTICIPANTS, axis=0)
df1 = read_group_5()
df2 = read_group_7()


#pd.concat([df1,df2.rename(columns={'b':'a'})], ignore_index=True)

""""
ratings = pd.DataFrame(list(zip(val_means, ar_means)), columns=["val", "ar"])
ratings["val_z"] = stats.zscore(ratings["val"])
ratings["ar_z"] = stats.zscore(ratings["ar"])
sns.lmplot(x="val_z", y="ar_z", data=ratings, order=2)
plt.tight_layout()
plt.show()

"""
df1
# Rename all valence/Liking scores 
df2 = df2.rename(columns=lambda x: re.sub('^LL0','VA7_0',x))
df1 = df1.rename(columns=lambda x: re.sub('^LN0','VA5_0',x))
df = df.rename(columns=lambda x: re.sub('^VA0','VA6_0',x))

# Rename all arousal/striking scores
df2 = df2.rename(columns=lambda x: re.sub('^SL0','AR7_0',x))
df1 = df1.rename(columns=lambda x: re.sub('^SN0','AR5_0',x))
df = df.rename(columns=lambda x: re.sub('^AR0','AR6_0',x))


#Rename BFI-scores Group 6
df = df.rename(columns=lambda x: re.sub('^BF02','BF01',x))

#Rename Song Knowledge in all groups
df2 = df2.rename(columns=lambda x: re.sub('^SK0','SK7_0',x))
df1 = df1.rename(columns=lambda x: re.sub('^SK0','SK5_0',x))
df = df.rename(columns=lambda x: re.sub('^SK0','SK6_0',x))

#rename group 7 native language
df2 = df2.rename(columns={"D004": "D012"})

#drop unimportant columns
df2 = df2.loc[:,~df2.columns.str.contains('^HI')]
df1 = df1.loc[:,~df1.columns.str.contains('^HI')]
df1 = df1.loc[:,~df1.columns.str.contains('^TIME')]
df2 = df2.loc[:,~df2.columns.str.contains('^TIME')]

new_df = pd.concat(
    [
        df1,
        df2,
        df
    ],
    ignore_index=True,
)

#Rename descriptive colums in all groups
new_df = new_df.rename(
    columns={
        "D001": "sex", 
        "D002_01":"age",
        "D017": "highest_edu",
        "D012": "native_language",
        "DD01": "knowdylan",
        "DD02": "familiar_dylan",
        "DD04": "wellknown_dylan",
        "DD05_01": "liking_gen_dylan",
    }
)


new_df["age"] = new_df["age"].astype("int32")
new_df["wellknown_dylan"] = new_df["wellknown_dylan"].astype("int32")
new_df["liking_gen_dylan"] = new_df["liking_gen_dylan"].astype("int32")

print(new_df["wellknown_dylan"])

sex = {1:"female", 2:"male"}
highest_edu = {2:"Highschool", 3 :"Vocational degree", 4 :"Bachelors", 5 :"Masters", 6 :"Doctorate"}
native_language = {1:"English", 2:"German", 3:"English-German Bilingual", 4 :"Other"}


new_df['sex'].replace("1", "female", inplace=True)
new_df['sex'].replace(1, "female", inplace=True)

new_df['sex']= new_df['sex'].replace("2", "male")
new_df['sex']= new_df['sex'].replace(2, "male")


for key, value in highest_edu.items():
    new_df['highest_edu'].replace(str(key), value, inplace=True)
    new_df['highest_edu'].replace(key, value, inplace=True)


for key, value in native_language.items():
    new_df['native_language'].replace(str(key), value, inplace=True)
    new_df['native_language'].replace(key, value, inplace=True)

    
    
print(list(new_df.columns))
new_df


new_df.to_excel('test.xlsx',  engine='openpyxl')

0      2
1      3
2      1
3      2
4      1
      ..
115    1
116    2
117    2
118    3
119    2
Name: wellknown_dylan, Length: 120, dtype: int32
['CASE', 'SERIAL', 'QUESTNNR', 'VA5_01_01', 'VA5_01_02', 'VA5_01_03', 'VA5_01_04', 'VA5_01_05', 'VA5_01_06', 'VA5_01_07', 'VA5_01_08', 'VA5_01_09', 'VA5_01_10', 'VA5_01_11', 'VA5_01_12', 'VA5_01_13', 'VA5_01_14', 'VA5_01_15', 'VA5_01_16', 'VA5_01_17', 'VA5_01_18', 'VA5_01_19', 'VA5_01_20', 'VA5_01_21', 'VA5_01_22', 'VA5_01_23', 'VA5_01_24', 'VA5_01_25', 'VA5_01_26', 'VA5_01_27', 'VA5_01_28', 'VA5_01_29', 'VA5_01_30', 'VA5_01_31', 'VA5_01_32', 'VA5_02_01', 'VA5_02_02', 'VA5_02_03', 'VA5_02_04', 'VA5_02_05', 'VA5_02_06', 'VA5_02_07', 'VA5_02_08', 'VA5_02_09', 'VA5_02_10', 'VA5_02_11', 'VA5_02_12', 'VA5_02_13', 'VA5_02_14', 'VA5_02_15', 'VA5_02_16', 'VA5_02_17', 'VA5_02_18', 'VA5_02_19', 'VA5_02_20', 'VA5_02_21', 'VA5_02_22', 'VA5_02_23', 'VA5_02_24', 'VA5_02_25', 'VA5_02_26', 'VA5_02_27', 'VA5_02_28', 'VA5_02_29', 'VA5_02_30', 'VA5_02_31', 'V